<a href="https://colab.research.google.com/github/p-perrone/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import xarray as xr

In [3]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

# Part 1: loading data

In [4]:
# this is for getting rid of edges (usually artifacted)
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

# testing geometry
svalbard_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',
    geodesic=False
)


# Part 2: Defining Olga Strait geometry and filtering data

In [5]:

# Define Olga Strait geometry (replace with actual coordinates)
olgastretet_geom = ee.Geometry.Polygon(
    [[[0, 80], [30, 80], [30, 78], [0, 78], [0, 80]]]
)

Sentinel-1 does not operate in a single, fixed mode all the time. It switches between modes (like IW, EW, SM, WV) based on a pre-defined mission plan. These modes have different default polarization configurations.
* Interferometric Wide Swath (IW): The most common mode over land. It is typically programmed to acquire in VV+VH (most common) or HH+HV.
* Extra Wide Swath (EW): Used over large areas like oceans and sea ice. It is often programmed for HH+HV or sometimes VV+VH.
* Wave Mode (WV): Used for sampling ocean waves. It is almost always VV only.
* Stripmap Mode (SM): Offers higher resolution and can be programmed for single (HH or VV) or dual (HH+HV or VV+VH) polarization.

For this work, based on existing literature, we focus on HH, HV, HH+HV with EW.

In [8]:
polarizations = ['HH', 'HV']
svalbard_s1_dict = {}


In [9]:

# Sentinel-1 collection for Svalb

for pol in polarizations:
    svalbard_s1_dict[pol] = (

    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'EW'))
    .select(pol)
    .map(mask_edge)
    )

    # https://www.sciencedirect.com/science/article/pii/S0034425724002116?via%3Dihub


In [10]:
svalbard_s1_pol = [
svalbard_s1HH := svalbard_s1_dict['HH'],
svalbard_s1HV := svalbard_s1_dict['HV']]

In [11]:
# time ranges
spring = ee.Filter.date('2023-03-01', '2023-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-06-10')
summer = ee.Filter.date('2025-06-11', '2025-08-31')

seasons = [spring, late_spring, summer]

In [12]:

Map = geemap.Map()
Map.centerObject(olgastretet_geom, zoom=6)
Map.addLayer(svalbard_s1HH, {'min': -25, 'max': 5}, 'Sentinel-1') # display a certain range of dB
Map

Map(center=[79.33695198759294, 14.999999999999998], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
# building the Sentinel-1 polarizations-seasons dataframe
season_names = ['spring', 'late_spring', 'summer']

svalbard_s1 = pd.DataFrame(
    index=polarizations,
    columns=season_names,
)

for pol in polarizations:
    for i, season_filter in enumerate(seasons):

        filtered_collection = svalbard_s1_dict[pol].filter(season_filter)

        svalbard_s1.loc[pol, season_names[i]] = filtered_collection

display(svalbard_s1)

,spring,late_spring,summer
HH,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."
HV,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."


In [14]:
svalbard_s1_spring = [
svalbard_s1HH_spring := svalbard_s1.loc['HH', 'spring'],
svalbard_s1HV_spring := svalbard_s1.loc['HV', 'spring']]

svalbard_s1_latespring = [
svalbard_s1HH_late_spring := svalbard_s1.loc['HH', 'late_spring'],
svalbard_s1HV_late_spring := svalbard_s1.loc['HV', 'late_spring']]

svalbard_s1_summer = [
svalbard_s1HH_summer := svalbard_s1.loc['HH', 'summer'],
svalbard_s1HV_summer := svalbard_s1.loc['HV', 'summer']]

In [21]:
"GEMINI HELPED"

def dates_with_data(collection, geometry):
    """ Finds the dates in which layers of a collection exist in a given geometry.

        Parameters:
        :: collection = layers collection
        :: geometry   = area of interest

        Returns:
        :: dates = list of dates with data (as 'yyyy-MM-ddTHH:mm:ss' strings)
    """

    # filter the collection by geometry
    filtered_collection = collection.filterBounds(geometry)

    # get a list of all image dates in the filtered collection
    # system:time_start is an attribute of every image (= starting time of aquisition)
    image_dates = ee.List(filtered_collection.aggregate_array('system:time_start'))

    # Check if there are any images
    if image_dates.size().getInfo() > 0:
        # Convert timestamps to dates and format them as strings
        dates = image_dates.map(lambda time: ee.Date(time).format(None, 'GMT'))
        return dates.sort().getInfo()
    else:
        return None

# Find the dates with data for HH polarization in the spring collection
# Define the sea ice geometry using the provided diagonal vertices
# Coordinates are in EPSG:3857
sea_ice_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',  # Specify the CRS here
    geodesic=False # Set to False for projected coordinates
)

dates_spring_HH = dates_with_data(svalbard_s1HH_spring, sea_ice_geom)
dates_spring_HV = dates_with_data(svalbard_s1HV_spring, sea_ice_geom)

dates_spring = [dates_spring_HH, dates_spring_HV]
polarizations = ['HH', 'HV']

# Print the dates found
for dates, pol in zip(dates_spring, polarizations):
  if dates:
      print(f"Dates with {pol} data in spring: {dates}")
  else:
      print(f"No {pol} data found in spring for the specified geometry.")

Dates with HH data in spring: ['2023-03-01T06:31:39', '2023-03-02T05:34:04', '2023-03-04T05:17:35', '2023-03-05T05:58:44', '2023-03-07T05:42:15', '2023-03-08T06:23:24', '2023-03-09T05:25:49', '2023-03-10T06:06:58', '2023-03-12T05:50:29', '2023-03-13T06:31:39', '2023-03-14T05:34:03', '2023-03-16T05:17:34', '2023-03-17T05:58:44', '2023-03-19T05:42:15', '2023-03-20T06:23:24', '2023-03-21T05:25:49', '2023-03-22T06:06:58', '2023-03-24T05:50:30', '2023-03-25T06:31:39', '2023-03-26T05:34:04', '2023-03-28T05:17:35', '2023-03-29T05:58:44', '2023-03-31T05:42:15', '2023-04-01T06:23:25', '2023-04-02T05:25:49', '2023-04-03T06:06:59', '2023-04-05T05:50:30', '2023-04-06T06:31:39', '2023-04-07T05:34:04', '2023-04-09T05:17:35', '2023-04-10T05:58:45', '2023-04-12T05:42:16', '2023-04-13T06:23:25', '2023-04-15T06:06:59', '2023-04-17T05:50:30', '2023-04-18T06:31:40', '2023-04-19T05:34:04']
Dates with HV data in spring: ['2023-03-01T06:31:39', '2023-03-02T05:34:04', '2023-03-04T05:17:35', '2023-03-05T05:58:

By using only HH and HV we make sure to have images of the same location always at the same time. Probably the acquisition method of S1 has always been EW in the last thre years (?)

In [28]:
svalbard_s1HH_spring

In [58]:
# Spring
SpringMap = geemap.Map()
SpringMap.centerObject(svalbard_geom, zoom=6)
spring_list = svalbard_s1HH_spring.toList(svalbard_s1HH_spring.size())

# Add mosaic layers for each polarization
for ic, pol in zip(svalbard_s1_spring, polarizations):
    ic_mosaic = ic.mosaic()
    SpringMap.addLayer(ic_mosaic, {'min': -25, 'max': 5}, f'{pol} Mosaic')

# Add a layer for a specific date (e.g., the date of the first image with HH data)
# Make sure dates_spring_HH has been populated by running the previous cell
if svalbard_s1HH_spring.size().getInfo() > 0:
    # Get the timestamp of the first image in the HH collection
    spring_image_hh = ee.Image(spring_list.get(0))
    first_date_timestamp_hh = spring_image_hh.get('system:time_start')

    # Create an ee.Date object from the timestamp
    first_date_hh = ee.Date(first_date_timestamp_hh)

    # Filter the HH collection by the first date
    ic_date_hh = svalbard_s1HH_spring.filterDate(first_date_hh, first_date_hh.advance(1, 'day')) # Filter for the whole day

    # Mosaic the images for the selected date
    mosaic_date_hh = ic_date_hh.mosaic()

    # Add the mosaicked image from the first date to the map
    SpringMap.addLayer(mosaic_date_hh, {'min': -25, 'max': 5}, f'HH on {first_date_hh.format("YYYY-MM-dd").getInfo()}')

# Add a layer for a specific date (e.g., the date of the first image with HV data)
if svalbard_s1HV_spring.size().getInfo() > 0:
    # Get the timestamp of the first image in the HV collection
    first_image_hv = svalbard_s1HV_spring.first()
    first_date_timestamp_hv = first_image_hv.get('system:time_start')

    # Create an ee.Date object from the timestamp
    first_date_hv = ee.Date(first_date_timestamp_hv)

    # Filter the HV collection by the first date
    ic_date_hv = svalbard_s1HV_spring.filterDate(first_date_hv, first_date_hv.advance(1, 'day')) # Filter for the whole day

    # Mosaic the images for the selected date
    mosaic_date_hv = ic_date_hv.mosaic()

    # Add the mosaicked image from the first date to the map
    SpringMap.addLayer(mosaic_date_hv, {'min': -25, 'max': 5}, f'HV on {first_date_hv.format("YYYY-MM-dd").getInfo()}')


SpringMap

Map(center=[78.32673233318731, 22.790622575802143], controls=(WidgetControl(options=['position', 'transparent_…

In [59]:
drawn_features = SpringMap.draw_last_feature
drawn_rectangle = drawn_features

if drawn_rectangle:
    # Get the geometry of the feature
    rectangle_geometry = drawn_rectangle.geometry()

    # Get the coordinates of the geometry as a Python object
    coordinates = rectangle_geometry.getInfo()['coordinates']

    # Print the coordinates
    print("Coordinates of the drawn rectangle:")
    print(coordinates)
else:
    print("Cannot extract coordinates as no rectangle feature is available.")

sea_ice_geom2 = ee.Geometry.Polygon(
    coordinates,
    geodesic=False # Set to False for projected coordinates
)

SpringMap.centerObject(sea_ice_geom2, zoom=6)
SpringMap

Cannot extract coordinates as no rectangle feature is available.


Map(center=[78.9677759610585, 28.68441149999989], controls=(WidgetControl(options=['position', 'transparent_bg…

In [68]:
# Function to sample an ImageCollection at the defined points
def sample_image_collection(collection, points):
    """ Samples an ImageCollection at given points.

        Parameters:
        :: collection = ImageCollection to sample
        :: points     = FeatureCollection of points

        Returns:
        :: sampled_data = FeatureCollection with sampled values and timestamp
    """
    sampled_data = collection.map(
        lambda image: image.reduceRegions(
            collection=points,
            reducer=ee.Reducer.first(), # Use ee.Reducer.mean() for mean value if needed
            scale=10 # Sentinel-1 resolution is 10m
        ).map(lambda feature: feature.set('timestamp', image.get('system:time_start'))) # Add timestamp
    ).flatten() # Flatten the collection of FeatureCollections into a single FeatureCollection

    return sampled_data

In [69]:
# Test the sample_image_collection function with an ImageCollection
# Using svalbard_s1HH_spring and random_points
sampled_spring_hh_test = sample_image_collection(svalbard_s1HH_spring, random_points)

# Print the sampled data (note: this will only show a small sample in the console)
# To get all data, you would typically export this FeatureCollection
print("Sampled data from svalbard_s1HH_spring (first 10 features):")
print(sampled_spring_hh_test.limit(10).getInfo())

Sampled data from svalbard_s1HH_spring (first 10 features):
{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [28.93377134291116, 78.94112487627362]}, 'id': 'S1A_EW_GRDM_1SDH_20230301T063139_20230301T063243_047447_05B241_43FF_0', 'properties': {'timestamp': 1677652299000}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [29.722022808810813, 78.854258845129]}, 'id': 'S1A_EW_GRDM_1SDH_20230301T063139_20230301T063243_047447_05B241_43FF_1', 'properties': {'timestamp': 1677652299000}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [29.35078403898252, 78.89617536261892]}, 'id': 'S1A_EW_GRDM_1SDH_20230301T063139_20230301T063243_047447_05B241_43FF_2', 'properties': {'timestamp': 1677652299000}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [28.712352918363475, 78.86995870805995]}, 'id': 'S1A_EW_GRDM_1SDH_20230301T063139_20230301T063243_047447_05B241_43FF_3', 'proper

In [76]:
# Define the number of random points to generate initially (significantly more than needed)
initial_num_points = 20

# Generate random points within the bounds of the sea ice geometry
random_points = ee.FeatureCollection.randomPoints(sea_ice_geom2, initial_num_points,
    seed=42)

sampled_data = spring_image_hh.sampleRegions(
    collection=random_points,
    properties=[],  # Add any specific properties you want
    scale=20,
    geometries=True  # Keep geometries for mapping
)
sampled_df = geemap.ee_to_df(sampled_data)

# Filter out rows with missing values (NaNs) that might result from sampling no-data areas
sampled_df = sampled_df.dropna()

# Display the first few rows of the DataFrame
print(sampled_df)

# Print the sampled data (note: this will only show a small sample in the console)
# To get all data, you would typically export this FeatureCollection
print(sampled_data.limit(10).getInfo())

SpringMap.addLayer(random_points, {'color': '00FF00'}, 'Sampled Points')
SpringMap.centerObject(sea_ice_geom2, 9) # Center map on the image
SpringMap
print(f"Number of features in sampled data: {sampled_data.size().getInfo()}")

Empty DataFrame
Columns: []
Index: []
{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['HH']}, 'features': []}
Number of features in sampled data: 0
